In [4]:
import os 
import numpy as np
from pathlib import Path

In [ ]:
res = Path("results/total_drücke.tyt")
res.touch()
nf = open(res,"w")
nf.write("p1,p2,p3,p4,p5,p6,p7,p8,p9,p10,p11,p12,13")

In [9]:
for i in os.listdir('Data'):
    path = 'Data/'+i
    with open(path, mode='r') as tf:
        dataex = np.genfromtxt((line.replace(',', '.') for line in tf),delimiter=";",skip_header=21).T
        data = np.zeros((2,len(dataex[3])))
        data[0],data[1] = dataex[3],dataex[4]
        sum = 0
        n=0
        for f in data[1]:
            sum+=f
            n+=1
        print(sum/n)
        

1.653491056910569e-08
3.299294634146342e-08
3.922776422764226e-08
4.53971219512195e-08
5.656794878048779e-08
6.628857723577233e-08
8.160305099778269e-08
1.1091749530956841e-07
1.3321933208255129e-07
1.5491302869440473e-07
1.8771477044476315e-07
2.2762053547671856e-07
2.685470750469041e-07
